In [ ]:
import torch 
from torch import nn

### Input Embeddings

In [ ]:
class InputEmbeddings(nn.Module):

    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        # (batch, seq_len) --> (batch, seq_len, d_model)
        # Multiply by sqrt(d_model) to scale the embeddings according to the paper
        return self.embedding(x) * math.sqrt(self.d_model)

![Alt Text](Pictures/InputEmbedding.PNG)


In [ ]:
#https://chatgpt.com/share/67b1af37-7db8-8008-b18d-5c629c5e2c8a

![Scaling](Pictures/WhyScaling.PNG)

### Postional Embedding

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        # Create a matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        # Create a vector of shape (seq_len)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # (seq_len, 1)
        # Create a vector of shape (d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * div_term) # sin(position * (10000 ** (2i / d_model))
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * div_term) # cos(position * (10000 ** (2i / d_model))
        # Add a batch dimension to the positional encoding
        pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        # Register the positional encoding as a buffer
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
        return self.dropout(x)

![](Pictures/Position1.PNG)

![](Pictures/Position2.PNG)

![](Pictures/Position3.PNG)

![](Pictures/Position4.PNG)

![](Pictures/postional5.PNG)

<p>Consider this that position is column vetor (seq,1) and divterm is (d_model/2) which is row vector that is halfed now column*row vector give the (seq,d_model/2) it give the half vector as we know there is sin and cosine function we passed both the these half data and concatenate at the alternative position   </p>

In [ ]:
def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
        return self.dropout(x)

<p> 
It simply truncated the postional embedding according to the given token size .. as sentence can be smaller than the max sequence lenght orr max token lenght 
</p>

### Layer Normalization

In [ ]:
class LayerNormalization(nn.Module):

    def __init__(self, features: int, eps:float=10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features)) # alpha is a learnable parameter
        self.bias = nn.Parameter(torch.zeros(features)) # bias is a learnable parameter

    def forward(self, x):
        # x: (batch, seq_len, hidden_size)
         # Keep the dimension for broadcasting
        mean = x.mean(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # Keep the dimension for broadcasting
        std = x.std(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # eps is to prevent dividing by zero or when std is very small
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

Open the markup to see the table  
`-1` determines the column. Let's say we have the following table:

|       | c1  | c2  | c3  |
|-------|-----|-----|-----|
| I     | 0.2 | 0.3 | 0.4 |
| am    | 0.2 | 0.4 | 0.4 |
| rizwan| 0.3 | 0.4 | 0.4 |

Since the mean is calculated column-wise (`dim=-1`), the result is:

|       | c1  |
|-------|-----|
| I     | 0.2 |
| am    | 0.3 |
| rizwan| 0.4 |


![](Pictures/LayerNormalization.PNG)

<p> Batch of n items each items has features . In layer normalization we calcuilate the mean and std of  each items</p>

![](Pictures/Layer2.PNG)

### Feed Forward Network

In [ ]:
class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # w1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # w2 and b2

    def forward(self, x):
        # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

### MultiHead Attention

In [ ]:

class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads
        
        # One word converted into 512 dims and 512 distrubute into 8 heads so one head will get 512/8= 64 dims
        # 64 dims of q,k,v will go through the linear transformation which is a neural network 
        # after this each 64 dims query will go dot product with 64 dims of keys 64 dot 64 = attention value 
        # Example attention scores for first head might look like:
        
            #         .....
            # Query shape is torch.Size([1, 10, 8, 64])
            # Key shape is torch.Size([1, 10, 8, 64])
            # Attention Score will be torch.Size([1, 10, 8, 8])
            
            # for every Q there is 64 dims
            # for every k there is 64 dims
            
            # so when we apply dot product we get a 64.dot(64) we get a single value 
            # so output attention will be  torch.Size([1, 10, 8, 8])
            # bwqd->bwkd 
            # after transpose bwqd->bwdk
            # after dot product 
            # bwqk
        
            #        i*64    am*64    a*64    cat*64
            # i*64    [0.2  0.3  0.1  0.4]
            # am*64   [0.1  0.4  0.2  0.3]
            # a*64    [0.1  0.2  0.3  0.4]
            # cat*64  [0.3  0.2  0.1  0.4]
            
        
        
        
        # after this we this we apply softmax to get the probality(sum of all equal to 1) of a query with 
        # respect to keys 

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(embed_size, embed_size)
        self.keys = nn.Linear(embed_size, embed_size)
        self.queries = nn.Linear(embed_size, embed_size)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask=None):
        # Get number of training examples
        N = query.shape[0] 
        # N No of batches
        # 

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        # Number of charater orr words

        
        values = self.values(values)  # (N, value_len, embed_size)
        keys = self.keys(keys)  # (N, key_len, embed_size)
        queries = self.queries(query)  # (N, query_len, embed_size)

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = queries.reshape(N, query_len, self.heads, self.head_dim)

        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just how I like doing matrix multiplication & bmm

        Q=queries
        K=keys
        V=values
        
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.embed_size)
        
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask==0, -1e9)
      
      
        attn_probs = torch.softmax(attn_scores, dim = -1)
        output = torch.matmul(attn_probs, V)
        # After this we multiply by the fc as in the picture below it is Wo
        output = self.fc_out(output)
        return output

![](Pictures/SelfAttention.png)

### Residual Connection

In [ ]:
class ResidualConnection(nn.Module):
    
        def __init__(self, features: int, dropout: float) -> None:
            super().__init__()
            self.dropout = nn.Dropout(dropout)
            self.norm = LayerNormalization(features)
    
        def forward(self, x, sublayer):
            return x + self.dropout(sublayer(self.norm(x)))

<p>Sub layer defines the layer in which the embedding goes after its output the x input that was previous given to the sublayer now appends to it</p>

### 